In [10]:
import os
import skimage
from skimage import io, color ,feature

In [11]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# -------------------- Utility function ------------------------
def normalize_label(str_):
    str_ = str_.replace(" ", "")
    str_ = str_.translate(str_.maketrans("","", "()"))
    str_ = str_.split("_")
    return ''.join(str_[:2])

def normalize_desc(folder, sub_folder):
    text = folder + " - " + sub_folder
    text = re.sub(r'\d+', '', text)
    text = text.replace(".", "")
    text = text.strip()
    return text

def print_progress(val, val_len, folder, sub_folder, filename, bar_size=10):
    progr = "#"*round((val)*bar_size/val_len) + " "*round((val_len - (val))*bar_size/val_len)
    if val == 0:
        print("", end = "\n")
    else:
        print("[%s] folder : %s/%s/ ----> file : %s" % (progr, folder, sub_folder, filename), end="\r")


In [13]:
dataset_dir = "/content/drive/My Drive/IIITNR_PROJECT/dataset"


In [14]:
os.listdir(dataset_dir)

['test', 'Train']

In [15]:
import cv2

In [16]:

import numpy as np
import cv2
import re

In [18]:


imgs = [] #list image matrix
labels = []
descs = []
for folder in os.listdir(dataset_dir):
    for sub_folder in os.listdir(os.path.join(dataset_dir, folder)):
        sub_folder_files = os.listdir(os.path.join(dataset_dir, folder, sub_folder))
        len_sub_folder = len(sub_folder_files) - 1
        for i, filename in enumerate(sub_folder_files):
            img = cv2.imread(os.path.join(dataset_dir, folder, sub_folder, filename))

            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

            h, w = gray.shape
            ymin, ymax, xmin, xmax = h//3, h*2//3, w//3, w*2//3
            crop = gray[ymin:ymax, xmin:xmax]

            resize = cv2.resize(crop, (0,0), fx=0.5, fy=0.5)

            imgs.append(resize)
            labels.append(sub_folder)
            descs.append(normalize_desc(folder, sub_folder))

            print_progress(i, len_sub_folder, folder, sub_folder, filename)




[##########] folder : test/Clay soil/ ----> file : images72.jpg
[##########] folder : test/Alluvial soil/ ----> file : image7 (2).jpeg
[##########] folder : test/Red soil/ ----> file : images202.jpg


[##########] folder : Train/Red soil/ ----> file : red3.jpg


In [19]:

from skimage.feature import graycomatrix, graycoprops

In [20]:
# ----------------- calculate greycomatrix() & greycoprops() for angle 0, 45, 90, 135 ----------------------------------
def calc_glcm_all_agls(img, label, props, dists=[5], agls=[0, np.pi/4, np.pi/2, 3*np.pi/4], lvl=256, sym=True, norm=True):

    glcm = graycomatrix(img,
                        distances=dists,
                        angles=agls,
                        levels=lvl,
                        symmetric=sym,
                        normed=norm)
    feature = []
    glcm_props = [propery for name in props for propery in graycoprops(glcm, name)[0]]
    for item in glcm_props:
            feature.append(item)
    feature.append(label)

    return feature



In [22]:
# Define the properties to calculate from GLCM
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

# Initialize a list to store GLCM features for all images
glcm_all_agls = []

# Loop through each image and label
for img, label in zip(imgs, labels):
    # Step 2: Apply thresholding
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)

    # Step 3: Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Step 4: Find the largest contour
    largest_contour = max(contours, key=cv2.contourArea)

    # Step 5: Get the bounding box of the largest contour
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Step 6: Crop the image using the bounding box
    cropped_image = img[y:y+h, x:x+w]

    # Call the GLCM calculation function
    glcm_all_agls.append(
        calc_glcm_all_agls(cropped_image, label, props=properties)
    )

ValueError: max() arg is an empty sequence

In [ ]:

# Define the column names
columns = []
angles = ['0', '45', '90', '135']
for name in properties:
    for ang in angles:
        columns.append(name + "_" + ang)

# Append the label column name
columns.append("label")

In [ ]:
# ----------------- call calc_glcm_all_agls() for all properties ----------------------------------
properties = ['dissimilarity', 'correlation', 'homogeneity', 'contrast', 'ASM', 'energy']

glcm_all_agls = []
for img, label in zip(imgs, labels):
    glcm_all_agls.append(
            calc_glcm_all_agls(img,
                                label,
                                props=properties)
                            )

columns = []
angles = ['0', '45', '90','135']
for name in properties :
    for ang in angles:
        columns.append(name + "_" + ang)

columns.append("label")


In [ ]:
import pandas as pd

# Create the pandas DataFrame for GLCM features data
glcm_df = pd.DataFrame(glcm_all_agls,
                      columns = columns)



In [ ]:
len(glcm_df)

1561

In [ ]:
#knowing dataset
type(glcm_df)
len(glcm_df)
glcm_df.shape

(1561, 25)

In [ ]:
 glcm_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1561 entries, 0 to 1560
Data columns (total 25 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   dissimilarity_0    1561 non-null   float64
 1   dissimilarity_45   1561 non-null   float64
 2   dissimilarity_90   1561 non-null   float64
 3   dissimilarity_135  1561 non-null   float64
 4   correlation_0      1561 non-null   float64
 5   correlation_45     1561 non-null   float64
 6   correlation_90     1561 non-null   float64
 7   correlation_135    1561 non-null   float64
 8   homogeneity_0      1561 non-null   float64
 9   homogeneity_45     1561 non-null   float64
 10  homogeneity_90     1561 non-null   float64
 11  homogeneity_135    1561 non-null   float64
 12  contrast_0         1561 non-null   float64
 13  contrast_45        1561 non-null   float64
 14  contrast_90        1561 non-null   float64
 15  contrast_135       1561 non-null   float64
 16  ASM_0              1561 

In [ ]:
glcm_df

,dissimilarity_0,dissimilarity_45,dissimilarity_90,dissimilarity_135,correlation_0,correlation_45,correlation_90,correlation_135,homogeneity_0,homogeneity_45,...,contrast_135,ASM_0,ASM_45,ASM_90,ASM_135,energy_0,energy_45,energy_90,energy_135,label
0,24.141694,25.572342,26.011782,25.984342,0.160142,0.072171,0.041680,0.042701,0.044852,0.042222,...,1142.727068,0.000173,0.000169,0.000169,0.000169,0.013159,0.013007,0.012989,0.013012,Red soil
1,25.124429,26.607912,27.314094,27.290155,0.212591,0.131703,0.087046,0.091550,0.041760,0.038626,...,1217.970812,0.000143,0.000140,0.000138,0.000139,0.011964,0.011812,0.011766,0.011769,Red soil
2,25.124429,26.607912,27.314094,27.290155,0.212591,0.131703,0.087046,0.091550,0.041760,0.038626,...,1217.970812,0.000143,0.000140,0.000138,0.000139,0.011964,0.011812,0.011766,0.011769,Red soil
3,24.141694,25.572342,26.011782,25.984342,0.160142,0.072171,0.041680,0.042701,0.044852,0.042222,...,1142.727068,0.000173,0.000169,0.000169,0.000169,0.013159,0.013007,0.012989,0.013012,Red soil
4,21.085366,36.684982,42.747826,36.372711,0.858902,0.720834,0.634388,0.694166,0.210628,0.134561,...,3428.246337,0.008052,0.004122,0.002971,0.003800,0.089735,0.064201,0.054506,0.061645,Red soil
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,23.002656,24.306926,25.101636,25.122024,0.562975,0.497148,0.453289,0.459698,0.047658,0.046799,...,1155.472742,0.000148,0.000146,0.000144,0.000143,0.012147,0.012073,0.011992,0.011950,Alluvial soil
1557,16.966723,16.834458,16.529494,16.931054,0.148447,0.159239,0.181681,0.145310,0.058666,0.059603,...,462.577564,0.000346,0.000349,0.000347,0.000351,0.018597,0.018678,0.018631,0.018727,Alluvial soil
1558,40.456790,43.867547,29.026246,31.113144,0.232904,0.154266,0.610504,0.523791,0.048027,0.035205,...,1987.470867,0.000274,0.000256,0.000344,0.000367,0.016565,0.016015,0.018547,0.019146,Alluvial soil
1559,20.743029,23.274921,24.465181,23.461793,0.596128,0.508363,0.463050,0.500767,0.056915,0.049051,...,951.163028,0.000156,0.000146,0.000141,0.000146,0.012481,0.012064,0.011872,0.012072,Alluvial soil


In [ ]:
glcm_df.to_csv("/content/drive/My Drive/IIITNR_PROJECT/data.csv" ,index=False)